In [1]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output

In [ ]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret)

info = client.get_account()
bal = pd.DataFrame(info['balances'])


clear_output(wait=True)


In [2]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
rate = 30
interval = '5m'
limit = 1000

# start epoch till now
start_time = 1676627007000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600

In [3]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [4]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [ ]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
rawdf['Closed'] = True
print(rawdf)

In [ ]:
df = rawdf.copy()
df = df.drop(df.index[-1])
df['Close_Time'] = df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(df)

In [5]:
def indicators(df):
#     bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()
#     rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()
    return df

In [13]:
def conditions(df):

    # c1 下影線 大於 實體Ｋ & 下影線 大於 上影線
    # c2 收盤 小於 布林下
#     df['c1_1'] = (abs(df['Low'] - df['Close']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Close']) > abs(df['High'] - df['Open']))
#     df['c1_2'] = (abs(df['Low'] - df['Open']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Open']) > abs(df['High'] - df['Close']))
    df['c2'] = df['Close'] > df['bb_l']

    # 條件達成
#     if df.iloc[-1].c1_1 & df.iloc[-1].c1_2 & df.iloc[-1].c2 == True
    df['signal'] = False
    df.loc[df.c2 , 'signal'] = True
            
    # 下一根進場
    
    df['openbuy'] = False
#     df.loc[df['signal'].shift() == True, 'openbuy'] = True
    return df

In [ ]:
def take_order(df)
    df.loc[df.index[-1], 'openbuy'] = True
    close_val = df['Close']
    low_val = df['Low']
    df['entry_p'] = close_val.shift(1)
    df['stop_loss'] = low_val.shift(1)
    in_position = True
    stop_loss = low_val.shift(1).at[index]

In [33]:
def on_open(ws):
    ws.send(data)

current_k = 0
in_position = False

def on_message(ws, message):
    global df, timezone, current_k
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    if df['Close_Time'][len(df) - 1] != current_k:
        print('time changed')
        current_k = df['Close_Time'][len(df) - 1]
        if df.iloc[df.index[-2]]['signal'] == True:
            print('openbuy')
            df.loc[df.index[-1], 'openbuy'] = True
            print(df.tail(3))
            close_val = df['Close']
            low_val = df['Low']
            df['entry_p'] = close_val.shift(1)
            df['stop_loss'] = low_val.shift(1)
            in_position = True
            stop_loss = low_val.shift(1)

    
    
    
#     out = json.loads(message)    
#     out = pd.DataFrame({'Close_Time':pd.to_datetime(out['E'], unit = 'ms') + pd.Timedelta(hours=timezone),
#                         'Open':float(out['k']['o']),
#                         'Close':float(out['k']['c']),
#                         'High':float(out['k']['h']),
#                         'Low':float(out['k']['l']),
#                         'Volume':float(out['k']['v']),
#                         'Closed':bool(out['k']['x'])},
#                         index=[pd.to_datetime(out['E'], unit = 'ms')])
    
#     out['Close_Time'] = out['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

#     if out['Closed'].iloc[0] == True:
#         df = pd.concat([df, out], axis = 0)
#         indicators(df)
#         conditions(df)
#         df = df.drop(df.index[-2])
#         if df.iloc[-1]['signal'].iloc[0] == True:
#             print('buy!!!!!!')

#     elif out['Closed'].iloc[0] == False & df.iloc[-1]['Closed'] == False:
#         df = df.drop(df.index[-1])
#         df = pd.concat([df, out], axis = 0)
#         indicators(df)
#         conditions(df)

#         df.iloc[-1] = out

#         df.iloc[-1] = out
#         indicators(df)
#         conditions(df)
#         print('nothing...')


    df = df.reset_index(drop=True)


    
#     if df.iloc[-2]['Closed'] == True:
#         if df.iloc[-2]['signal'] == True:
#             print('buy!!!!!!!!!!!!!!')
#     else:
#         if df.iloc[-2]['Closed'] == False:
#             df = df.drop(df.index[-2])
        

    

#     df.apply(conditions, axis = 1)
    
#     df = df.tail(50)
#     print(df.tail(3))

    
    # out
    # 檢查 out.closed 是否為 True
        # 是:
            # 檢查是否有 signal
                # 有:
                    #下個點位進場，設定進出場條件，接著新增資料
                # 否:
                    # 沒事，接著新增資料
        # 否:
            # 取代為新資料

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

time changed
openbuy
                      Close_Time     Open    Close     High      Low  \
69 2023-02-17 23:34:59.999000064  1666.45  1669.74  1671.22  1666.10   
70 2023-02-17 23:39:59.999000064  1669.74  1671.92  1676.00  1666.02   
71 2023-02-17 23:44:59.999000064  1671.93  1677.32  1677.47  1670.50   

       Volume         bb_u         bb_m         bb_l        rsi    c2  signal  \
69  1672.4813  1677.170100  1665.036667  1652.903234  53.494175  True    True   
70  4594.0826  1677.652459  1665.270333  1652.888208  55.390986  True    True   
71  1851.8674  1678.790363  1665.674667  1652.558970  59.768294  True    True   

    openbuy  
69    False  
70    False  
71     True  


In [ ]:
df = rawdf

In [ ]:
buydf = conditions(df)

# 進出場

in_position = False
stop_loss = np.nan

for index, row in buydf.iterrows():

    if index == 0:
        continue
        
        # set entry price (前一根Ｋ收盤價進場)
        # set stop loss (前一根Ｋ線低點停損)
        # 進場
    elif buydf.at[index, 'openbuy'] == True:
        close_val = buydf['Close']
        low_val = buydf['Low']
        buydf.at[index, 'entry_p'] = close_val.shift(1).at[index]
        buydf.at[index, 'stop_loss'] = low_val.shift(1).at[index]
        buydf.at[index, 'position'] = 'Buy'
        in_position = True
        stop_loss = low_val.shift(1).at[index]
    

    # 吃筍
    #-----------------------------重要-----------------------------
    # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
    elif in_position == True and buydf.at[index, 'Close'] < stop_loss:
        buydf.at[index, 'position'] = 'Stop'
        in_position = False
        stop_loss = np.nan

    # set take profit （高點碰到上布林，即刻出場）
    elif buydf.at[index, 'High'] >= buydf.at[index, 'bb_u'] and in_position == True:

        buydf.at[index, 'position'] = 'Sell'
        in_position = False
        stop_loss = np.nan


# 過濾有訊號或事件發生的Ｋ線
buydf = buydf[(buydf['openbuy'] == True) |
              (buydf['signal'] == True) | 
              (buydf['position'] == 'Buy') |
              (buydf['position'] == 'Sell') |
              (buydf['position'] == 'Stop')]


print(buydf)


In [ ]:
# 部位回測

posdf = buydf
posdf = posdf.reset_index(drop = True)
posdf = posdf[(posdf['position'] == 'Buy') |
              (posdf['position'] == 'Sell') |
              (posdf['position'] == 'Stop')]

# 一次進場多少單位
pos_size = 1

col = ['Close_Time', 'Open', 'Close', 'High', 'Low', 'sma', 'bb_u', 'bb_l', 'rsi', 'position','entry_p', 'stop_loss']
pos = posdf[col]
pos = pos.reset_index(drop = True)


for index, row in pos.iterrows():
    
    current_pos = 0
    
    # 進場
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'size'] = pos_size
        pos.exit_p = np.nan
    
    # 出場
    if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        
        #-----------------------------重要-----------------------------
        # 實戰需即刻出場
        
        # 停利：打到上布林
        if pos.at[index, 'position'] == 'Sell':
            pos.at[index, 'exit_p'] = pos.at[index, 'bb_u'] * 1

        # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
        if pos.at[index, 'position'] == 'Stop':
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
                break

        # 計算每次出場部位大小（每次出場皆清倉）
        for i in range(index -1, -1, -1):
            if pos.at[i, 'position'] == 'Buy':
                current_pos += pos.at[i, 'size']
                if i == 0:
                    pos.at[index, 'size'] = -current_pos
                else:
                    continue
            else:
                pos.at[index, 'size'] = -current_pos
                current_pos = 0
                break


# 計算部位價值
for index, row in pos.iterrows():
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
    elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)

    
        
print(pos)
pos.to_csv('pos.csv')


# 手續費、滑點、價差
fee = 0.075 / 100
amt_abs_sum = pos.amt.abs().sum()
print(amt_abs_sum)


# 損益
leverage = 10
ttl_profit = -pos.amt.sum()


# 計算進場最大部位來代表總進場成本，以計算報酬率
pos['consec_entry'] = (pos['position'] != pos['position'].shift()).cumsum()
group_consec = pos.groupby('consec_entry').apply(lambda x:x.loc[x['position'] == 'Buy', 'amt' ].sum())
max_entry = group_consec.max()

profit_per = "{:.2f}%".format(ttl_profit / (max_entry/leverage) * 100)
        
print(ttl_profit, max_entry, profit_per)



In [ ]:
plt.title(symbol + ' SMA_' + str(rate))
plt.xlabel('time')
plt.ylabel('Closing Prices')
plt.plot(df.Close, label='Closing Prices')
plt.plot(df.sma, label= str(rate) + ' SMA')
plt.plot(df.bb_u, label='Bollinger Up', c='g')
plt.plot(df.bb_l, label='Bollinger Down', c='r')
plt.legend()
plt.show()